# Robotics, Vision & Control 3e: for Python
## Chapter 8: Manipulator Velocity

Copyright (c) 2021- Peter Corke

In [ ]:
try:
    from google.colab import output
    print('Running on CoLab')
    output.enable_custom_widget_manager()
    !pip install ipympl
    !pip install roboticstoolbox-python>=1.0.2
    !pip install --no-deps rvc3python
    COLAB = True

except ModuleNotFoundError:
    COLAB = False

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"
from IPython.display import HTML

%matplotlib widget
import matplotlib.pyplot as plt

# add RTB examples folder to the path
import sys, os.path
import RVC3 as rvc
sys.path.append(os.path.join(rvc.__path__[0], 'models'))

# helper function to run bdsim in a subprocess and transfer results using a pickle file
import pickle
def run_shell(tool, **params):
    global out
    pyfile = os.path.join(rvc.__path__[0], "models", tool+".py")
    cmd = f"python {pyfile} -H +a -o"
    for key, value in params.items():
        cmd += f' --global "{key}={value}"'
    print(cmd)
    os.system(cmd)
    with open("bd.out", "rb") as f:
        out = pickle.load(f)

# ------ standard imports ------ #
import numpy as np
from scipy import linalg
import math
from math import pi
np.set_printoptions(
    linewidth=120, formatter={
        'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})
np.random.seed(0)
from spatialmath import *
from spatialmath.base import *
from roboticstoolbox import *


# 8.1 Manipulator Jacobian


## 8.1.1 Jacobian in the World Coordinate Frame


In [ ]:
import sympy
a1, a2 = sympy.symbols("a1, a2")
e = ERobot2(ET2.R() * ET2.tx(a1) * ET2.R() * ET2.tx(a2))

In [ ]:
q = sympy.symbols("q:2")

In [ ]:
TE = e.fkine(q);

In [ ]:
p = TE.t

In [ ]:
J = sympy.Matrix(p).jacobian(q)
J.shape

In [ ]:
ur5 = models.URDF.UR5();
J = ur5.jacob0(ur5.q1)

In [ ]:
# ur5.teach(ur5.q1, backend="pyplot")
ur5.plot(ur5.q1, backend="pyplot")

A robot model created from a URDF file uses Swift by default for visualization.  We explicitly set the backed to `"plplot"` but that does not support teaching in the Jupyter environment.

## 8.1.2 Jacobian in the End-Effector Coordinate Frame


In [ ]:
ur5.jacobe(ur5.q1)

## 8.1.3 Analytical Jacobian


In [ ]:
rotvelxform((0.1, 0.2, 0.3), representation="rpy/xyz")

In [ ]:
ur5.jacob0_analytical(ur5.q1, "rpy/xyz");

# 8.2 Application: Resolved-Rate Motion Control


In [ ]:
if COLAB:
  %run -m RRMC -H -g  # no graphics
else:
  run_shell("RRMC")


In [ ]:
t = out.clock0.t;
q = out.clock0.x;

In [ ]:
xplot(t, q[:, :3], stack=True);

In [ ]:
from roboticstoolbox.models.DH import Puma560  # need to explicitly import this because we run RRMC in a subprocess
puma = Puma560()

Tfk = puma.fkine(q);

In [ ]:
xplot(out.clock0.t, Tfk.t, stack=True);

In [ ]:
if COLAB:
  %run -m RRMC2 -H -g  # no graphics
else:
  run_shell("RRMC2")


# 8.3 Jacobian Condition and Manipulability


## 8.3.1 Jacobian Singularities


In [ ]:
J = ur5.jacob0(ur5.qz)

In [ ]:
np.linalg.det(J)

In [ ]:
np.linalg.matrix_rank(J)

In [ ]:
jsingu(J)

In [ ]:
qns = np.full((6,), np.deg2rad(5))

In [ ]:
J = ur5.jacob0(qns);

In [ ]:
qd = np.linalg.inv(J) @ [0, 0, 0, 0.1, 0, 0]

In [ ]:
np.linalg.det(J)

In [ ]:
np.linalg.cond(J)

In [ ]:
qd = np.linalg.inv(J) @ [0, 0.1, 0, 0, 0, 0]

## 8.3.2 Velocity Ellipsoid and Manipulability


In [ ]:
planar2 = models.ETS.Planar2();

In [ ]:
# planar2.teach(np.deg2rad([30, 40]), vellipse=True);
planar2.plot(np.deg2rad([30, 40]), vellipse=True);

In [ ]:
J = ur5.jacob0(ur5.q1);

In [ ]:
Jt = J[:3, :];  # first 3 rows

In [ ]:
E = np.linalg.inv(Jt @ Jt.T)
plot_ellipsoid(E);

In [ ]:
e, _ = np.linalg.eig(E);
radii = 1 / np.sqrt(e)

In [ ]:
J = ur5.jacob0(np.full((6,), np.deg2rad(1)));
Jr = J[3:, :];  # last 3 rows
E = np.linalg.inv(Jr @ Jr.T);
plot_ellipsoid(E);
plt.gca().set_aspect("equal")

In [ ]:
e, x = np.linalg.eig(E);
radii = 1 / np.sqrt(e)

In [ ]:
x[:, 0]

In [ ]:
ur5.vellipse(qns, "rot");

In [ ]:
ur5.manipulability(ur5.q1)

In [ ]:
ur5.manipulability(ur5.qz)

In [ ]:
ur5.manipulability(ur5.qz, axes="both")

## 8.3.4 Dealing with a non-square Jacobian


### 8.3.4.1 Jacobian for Under-Actuated Robot


In [ ]:
planar2 = models.ETS.Planar2();
qn = [1, 1];

In [ ]:
J = planar2.jacob0(qn)

In [ ]:
xd_desired = [0.1, 0.2, 0];

In [ ]:
qd = np.linalg.pinv(J) @ xd_desired

In [ ]:
J @ qd

In [ ]:
np.linalg.norm(xd_desired - J @ qd)

In [ ]:
Jxy = J[:2, :];
qd = np.linalg.inv(Jxy) @ xd_desired[:2]

In [ ]:
xd = J @ qd

In [ ]:
np.linalg.norm(xd_desired - J @ qd)

### 8.3.4.2 Jacobian for Overactuated Robot


In [ ]:
panda = models.ETS.Panda();
TE = SE3.Trans(0.5, 0.2, -0.2) * SE3.Ry(pi);
# sol = panda.ikine_LMS(TE);
sol = panda.ikine_LM(TE);

In [ ]:
J = panda.jacob0(sol.q);
J.shape

In [ ]:
xd_desired = [0.1, 0.2, 0.3, 0, 0, 0];

In [ ]:
qd = np.linalg.pinv(J) @ xd_desired

In [ ]:
J @ qd

In [ ]:
np.linalg.matrix_rank(J)

In [ ]:
N = linalg.null_space(J);
N.shape
N.T

In [ ]:
np.linalg.norm( J @ N[:,0])

In [ ]:
qd_0 = [0, 0, 0, 0, 1, 0, 0];

In [ ]:
qd = N @ np.linalg.pinv(N) @ qd_0

In [ ]:
np.linalg.norm(J @ qd)

# 8.4 Force Relationships


## 8.4.1 Transforming Wrenches to Joint Space


In [ ]:
tau = ur5.jacob0(ur5.q1).T @ [0, 20, 0, 0, 0, 0]

In [ ]:
tau = ur5.jacob0(ur5.q1).T @ [20, 0,  0, 0, 0, 0]

## 8.4.2 Force Ellipsoids


In [ ]:
# planar2.teach(np.deg2rad([30, 40]), fellipse=True);
planar2.plot(np.deg2rad([30, 40]), fellipse=True);

# 8.6 Advanced Topics


## 8.6.1 Manipulability Jacobian


In [ ]:
panda = models.ETS.Panda()
panda.jacobm(panda.qr).T